<a href="https://colab.research.google.com/github/pigsoft/Test/blob/master/PyTorch_CNN_(separate).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import torch
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms

torch.manual_seed(0)
dtype = torch.double
device = torch.device("cpu")
# device = torch.device("cuda")

In [0]:
# class all_activation_function:
#     def __init__(self):
#         pass
        
#     def Relu(self, x) :
#         return x*(x>0)
    
#     def d_Relu(self, x) :
#         return 1.0*(x>0)
    
#     def PRelu(self, x, alpha) :
#         return x*(x>=0) + alpha*x*(x<0)
    
#     def d_PRelu(self, x, alpha) :
#         return 1.0*(x>=0) + alpha*(x<0)
    
#     def Tanh(self, x) :
#         return torch.tanh(x)
    
#     def d_Tanh(self, x) :
#         return 1 - torch.tanh(x)**2
    
#     def sigmoid(self, x) :
#         return 1.0/(1 + torch.exp(-x))
    
#     def d_sigmoid(self, x) :
#         tmp = torch.exp(-x)
#         return tmp/((1-tmp)**2)
    
#     def Elu(self, x, alpha) :
#         return alpha*(torch.exp(x)-1)*(x<=0) + x*(x>0)
    
#     def d_Elu(self, x, alpha) :
#         return alpha * torch.exp(x) * (x<=0) + 1.0 * (x>0)
    
#     def softmax(self, x):
#         return torch.exp(x) / torch.reshape(torch.sum(torch.exp(x), axis=1), (-1, 1))
    
#     def log_softmax(self, x):
#         return torch.log(self.softmax(x))
    
#     def CrossEntropy(self, x, y):
#         log_softmax = -self.log_softmax(x)
        
#         expected_loss = torch.zeros(1, dtype=dtype)
#         for row, col in enumerate(y):
#             expected_loss += log_softmax[row, col]
#         expected_loss /= log_softmax.shape[0]

#         return expected_loss

In [0]:
class Linear:
    def __init__(self, in_features, out_features):
        self.in_features = in_features
        self.out_features = out_features
        self.W = torch.randn(in_features, out_features, device=device, dtype=dtype, requires_grad=True)
        self.b = torch.randn(out_features, device=device, dtype=dtype, requires_grad=True)
    
    def forward(self, X):
        return X @ W + b

In [0]:
class Conv2d:
    # input – input tensor of shape (minibatch,in_channels,iH,iW)
    # weight – filters of shape (out_channels, in_channels, kH, kW)
    # bias – optional bias tensor of shape (out_channels). Default: None
    # stride – the stride of the convolving kernel. Can be a single number or a tuple (sH, sW). Default: 1
    # padding – implicit paddings on both sides of the input. Can be a single number or a tuple (padH, padW). Default: 0

    # The parameters kernel_size, stride, padding, dilation can either be:
    #     a single int – in which case the same value is used for the height and width dimension
    #     a tuple of two ints – in which case, the first int is used for the height dimension, and the second int for the width dimension

    # In the simplest case, the output value of the layer with input size (N, C_in, H, W) and output (N, C_out, H_out, W_out)
    # weight – filters of shape (C_out, C_in, kH, kW) 

    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        self.in_channels  = in_channels
        self.out_channels = out_channels
        self.kernel_size  = kernel_size
        self.stride       = stride
        self.padding      = padding

        if isinstance(kernel_size, int):
            self.kernel_size = (kernel_size, kernel_size)
        if isinstance(stride, int):
            self.stride = (stride, stride)
        
        self.W = torch.randn(self.out_channels, self.in_channels, self.kernel_size[0], self.kernel_size[1], device=device, dtype=dtype, requires_grad=True)
        self.b = torch.randn(self.out_channels, device=device, dtype=dtype, requires_grad=True)

    def forward(self, X):
        if self.padding > 0:
            # Add padding
            left_right = torch.zeros((X.shape[0], self.padding))
            X = torch.cat((left_right, X, left_right), 1)
            top_down   = torch.zeros((self.padding, X.shape[1]))
            X = torch.cat((top_down, X, top_down), 0)


        print('conv2d PyTorch')
        print(F.conv2d(X.clone(), self.W.clone(), bias=self.b.clone()))
        print()

        X_minibatch = X.shape[0]
        X_dim = X.shape[1]
        X_row = X.shape[2]
        X_col = X.shape[3]

        X_out_minibatch = X.shape[0]
        X_out_dim = self.out_channels
        X_out_row = int((X_row - self.kernel_size[0] + 2 * self.padding) / self.stride[0] + 1)
        X_out_col = int((X_col - self.kernel_size[1] + 2 * self.padding) / self.stride[1] + 1)

        X_out = torch.zeros((X_out_minibatch, X_out_dim, X_out_row, X_out_col))

        for n in range(X_out_minibatch):
            for f in range(X_out_dim):
                for i in range(self.kernel_size[0], X_row + 1, self.stride[0]):
                    for j in range(self.kernel_size[1], X_col + 1, self.stride[1]):
                        # print('({}, {})'.format(i, j))
                        # print('X')
                        # print(X[n, :, i - self.kernel_size[0], j - self.kernel_size[1]])
                        # print('W')
                        # print(self.W[f, ...])
                        X_out_i = torch.sum(X[n, :, i - self.kernel_size[0]: i, j - self.kernel_size[1]: j] * self.W[f, ...]) + self.b[f]
                        X_out[n][f][i - self.kernel_size[0]][j - self.kernel_size[1]] = X_out_i

        # # i-th mini-batch
        # for id_X, X_i in enumerate(X):
        #     # i-th filter
        #     for id_W, W_i in enumerate(self.W):
        #         # Convolute X_i and i-th filter
        #         print("X_i")
        #         print(X_i.shape)
        #         print(X_i)
        #         print("W_i")
        #         print(W_i.shape)
        #         print(W_i)
        #         for i in range(self.kernel_size[0], X_row + 1, self.stride[0]):
        #             for j in range(self.kernel_size[1], X_col + 1, self.stride[1]):
        #                 print('i =', i - self.kernel_size[0], i)
        #                 print('j =', j - self.kernel_size[1], j)
        #                 print(X_i[i - self.kernel_size[0]: i, j - self.kernel_size[1]: j] * W_i)
        #                 X_out_i = torch.sum(X_i[i - self.kernel_size[0]: i, j - self.kernel_size[1]: j] * W_i) + b[id_W]
        #                 X_out[id_X][id_W][i - self.kernel_size[0]][j - self.kernel_size[1]] = X_out_i     
        return X_out

In [0]:
# class MaxPool2d:
#     def __init__(self, X, kernel_size, stride=0, padding=0):
#         self.kernel_size  = kernel_size
#         self.stride       = stride
#         self.padding      = padding

#         if isinstance(kernel_size, int):
#             kernel_size = (kernel_size, kernel_size)
#         if isinstance(stride, int):
#             stride = (stride, stride)

#     def forward(self, X):
#         if slef.padding > 0:
#             # Add padding
#             left_right = torch.zeros((X.shape[0], padding))
#             X = torch.cat((left_right, X, left_right), 1)
#             top_down   = torch.zeros((padding, X.shape[1]))
#             a = torch.cat((top_down, X, top_down), 0)
        
#         X_minibatch = X.shape[0]
#         X_dim = X.shape[1]
#         X_row = X.shape[2]
#         X_col = X.shape[3]

#         X_out_minibatch = X.shape[0]
#         X_out_dim = X.shape[1]
#         X_out_row = int((X_row - kernel_size[0] + 2 * padding) / stride[0] + 1)
#         X_out_col = int((X_col - kernel_size[1] + 2 * padding) / stride[1] + 1)

#         X_out = torch.zero((X_out_minibatch, X_out_dim, X_out_row, X_out_col))
        
#         # i-th mini-batch
#         for id_X, X_i in enumerate(X):
#             # i-th dimension
#             for id_X_dim in range(X_dim):
#                 # max_pool2d
#                 for i in range(kernel_size[0], X_row + 1, self.stride[0]):
#                     for j in range(kernel_size[1], X_col + 1, self.stride[1]):
#                         X_out_i = torch.sum(X_i[i - kernel_size[0]: i, j - kernel_size[1]: j] * W_i) + b[id_W]
#                         X_out[id_X][id_X_dim][i - kernel_size[0]][j - kernel_size[1]] = X_out_i     
#         return X_out

In [0]:
class nn:
    def __init__(self):
        self.weight = []

In [7]:
# Test Conv2d
A = torch.arange(2.0 * 1.0 * 3.0 * 3.0, dtype=dtype).reshape(2, 1, 3, 3)
print(A)
print()
conv2d = Conv2d(1, 2, 2)

print("Convolute")
conv1 = conv2d.forward(A)
print(conv1.shape)
print(conv1)

tensor([[[[ 0.,  1.,  2.],
          [ 3.,  4.,  5.],
          [ 6.,  7.,  8.]]],


        [[[ 9., 10., 11.],
          [12., 13., 14.],
          [15., 16., 17.]]]], dtype=torch.float64)

Convolute
conv2d PyTorch
tensor([[[[ -5.2753,  -5.6381],
          [ -6.3637,  -6.7265]],

         [[  2.7602,   1.5185],
          [ -0.9650,  -2.2068]]],


        [[[ -8.5404,  -8.9032],
          [ -9.6288,  -9.9916]],

         [[ -8.4155,  -9.6572],
          [-12.1407, -13.3825]]]], dtype=torch.float64,
       grad_fn=<ThnnConv2DBackward>)

torch.Size([2, 2, 2, 2])
tensor([[[[ -5.2753,  -5.6381],
          [ -6.3637,  -6.7265]],

         [[  2.7602,   1.5185],
          [ -0.9650,  -2.2068]]],


        [[[ -8.5404,  -8.9032],
          [ -9.6288,  -9.9916]],

         [[ -8.4155,  -9.6572],
          [-12.1407, -13.3825]]]], grad_fn=<CopySlices>)


In [0]:
# class neural_network(all_activation_function):

#     def __init__(self, X, y):
#         self.X_train = torch.from_numpy(X).to(device=device)
#         self.X = torch.from_numpy(X).to(device=device)
#         self.y = torch.from_numpy(y)
#         self.W = []
#         self.b = []
#         self.n_node = []
#         self.activation_func = []
#         self.loss_func = None
#         self.list_loss = []
#         self.loss_ = None

#         self.para_conv = []
#         self.para_pool = []
#         self.para_flatten = []

#         self.order_node = []

#         self.dict_activation_func = {"sigmoid": self.sigmoid,
#                                      "softmax": self.softmax,
#                                      "softmax_cross_entropy": self.CrossEntropy
#                                      }

#     def linear(self, X, W, b):
#         Z = X @ W + b
#         return Z
    
#     def conv2d(self, X, W, b, filters, kernel_size, strides, padding):
#         if padding > 0:
#             # Add padding
#             left_right = torch.zeros((X.shape[0], padding))
#             X = torch.cat((left_right, X, left_right), 1)
#             top_down   = torch.zeros((padding, X.shape[1]))
#             a = torch.cat((top_down, X, top_down), 0)

#         X_dims = X.shape[0]
#         X_rows = X.shape[1]
#         X_cols = X.shape[2]
#         X_out = torch.tensor([])

#         X_out_dims = kernel_size
#         X_out_rows = int((X_rows - filters + 2 * padding) / strides + 1)
#         X_out_cols = int((X_cols - filters + 2 * padding) / strides + 1)

#         # i-th filter
#         for W_i in W:
#             # Convolute X and i-th filter
#             for i in range(kernel_size, X_rows + 1, strides):
#                 for j in range(kernel_size, X_cols + 1, strides):
#                     X_out_i = torch.sum(X[i - kernel_size: i, j - kernel_size: j] * W_i) + b
#                     X_out = torch.cat((X_out, X_out_i))

#         X_out = X_out.reshape(X_out_dims, X_out_rows, X_out_cols)
    
#     def maxpool2d(self, X, kernel_size, strides, padding=0):
#         if padding > 0:
#             # Add padding
#             left_right = torch.zeros((X.shape[0], padding))
#             X = torch.cat((left_right, X, left_right), 1)
#             top_down   = torch.zeros((padding, X.shape[1]))
#             a = torch.cat((top_down, X, top_down), 0)
        
#         X_dims = X.shape[0]
#         X_rows = X.shape[1]
#         X_cols = X.shape[2]
#         X_out = torch.tensor([])

#         X_out_dims = X_dims
#         X_out_rows = int((X_rows - filters + 2 * padding) / strides + 1)
#         X_out_cols = int((X_cols - filters + 2 * padding) / strides + 1)

#         # i-th filter
#         for W_i in W:
#             # Convolute X and i-th filter
#             for i in range(kernel_size, X_rows + 1, strides):
#                 for j in range(kernel_size, X_cols + 1, strides):
#                     X_out_i = torch.max(X[i - kernel_size: i, j - kernel_size: j])

#     def fc(self, n_node, activation_func):
#         self.n_node.append(n_node)
#         self.activation_func.append(activation_func)
    
#     def conv(self, filters, kernel_size, strides=1, padding=0, activation='Relu'):
        

#     def loss(self, n_node, loss_func):
#         self.n_node.append(n_node)
#         self.loss_func = loss_func
    
#     def forward_train(self, epoch):
#         # Set X_train to original value for forward 
#         self.X = self.X_train

#         # i = 0 (first layer) to len(self.n_node) - 1 
#         for i in range(len(self.n_node) - 1):
#             n_prev_node = self.X.shape[1]
#             n_curr_node = self.n_node[i]

#             Z = self.linear(self.X, self.W[i], self.b[i])
#             self.X = self.dict_activation_func[self.activation_func[i]](Z)
        
#         # Last layer (loss function)        
#         Z = self.linear(self.X, self.W[-1], self.b[-1])
#         self.loss = self.dict_activation_func[self.loss_func](Z, self.y)
#         self.list_loss.append(self.loss.data)
        
#         if epoch % 100 == 0:
#             print('epoch: {}, \tloss: {}'.format(epoch, self.loss.data[0]))
    
#     def forward_test(self):
#         # Set X_train to original value for forward 
#         self.X = self.X_test

#         # i = 0 (first layer) to len(self.n_node) - 1 
#         for i in range(len(self.n_node) - 1):
#             n_prev_node = self.X.shape[1]
#             n_curr_node = self.n_node[i]
            
#             Z = self.linear(self.X, self.W[i], self.b[i])
#             self.X = self.dict_activation_func[self.activation_func[i]](Z)
        
#         # Last layer (loss function)        
#         Z = self.linear(self.X, self.W[-1], self.b[-1])
#         predicted = torch.argmax(Z, axis=1)
#         return predicted

#     def backward(self, lr):
#         # Backpropagation
#         self.loss.backward()
#         for i in range(len(self.W)):
#             with torch.no_grad():
#                 self.W[i] -= self.W[i].grad * lr
#                 self.b[i] -= self.b[i].grad * lr

#                 self.W[i].grad.zero_()
#                 self.b[i].grad.zero_()

#     def fit(self, n_epoch=2000, lr=1e-6):
#         # Create weights and biases
#         n_prev_node = self.X_train.shape[1]
#         for i in range(len(self.n_node)):
#             n_curr_node = self.n_node[i]
            
#             torch.manual_seed(0)
#             W = torch.randn(n_prev_node, n_curr_node, device=device, dtype=dtype, requires_grad=True)
#             b = torch.randn(n_curr_node, device=device, dtype=dtype, requires_grad=True)
#             self.W.append(W)
#             self.b.append(b)

#             n_prev_node = self.n_node[i]
        
#         for epoch in range(n_epoch):
#             self.forward_train(epoch)
#             self.backward(lr)

#     def predict(self, X_test):
#         self.X_test = torch.from_numpy(X_test)
#         predicted = self.forward_test()
#         return predicted
        

In [0]:
# from sklearn.datasets import load_digits
# from sklearn.model_selection import train_test_split

# digits = load_digits()
# X = np.array(digits.data)
# y = np.array(digits.target)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
# print(X_train.shape)
# print(X_test.shape)
# print(y_train.shape)
# print(y_test.shape)
# print(type(X_train[0][0]))
# print(type(y_train[0]))

In [0]:
# nn = neural_network(X_train, y_train)
# nn.fc(10, 'sigmoid')
# nn.fc(10, 'sigmoid')
# nn.fc(10, 'sigmoid')
# nn.loss(10, 'softmax_cross_entropy')
# nn.fit(n_epoch=1001, lr=1.5)

In [0]:
# from sklearn.metrics import accuracy_score

# y_predicted = nn.predict(X_test)
# acc = accuracy_score(y_test, y_predicted)
# print('Accuracy:', acc * 100.0)

In [0]:
# torch.nn.Conv1d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
# torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
# torch.nn.Conv3d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')

In [0]:
# a = torch.tensor([[1., 2.], [3., 4.], [5., 6.]])
# # print(a)
# l = torch.zeros((a.shape[0], 1))
# print(l)
# a = torch.cat((l, a, l), 1)
# print(a)

# r = torch.zeros((1, a.shape[1]))
# a = torch.cat((r, a, r), 0)
# print(a)

In [0]:
# a = []
# b = [1, 2, 3]
# print(not a)
# print(not b)

In [0]:
# a = torch.tensor([[[1], [2]], [[3], [4]], [[5], [6]]])
# print(a.shape)
# print()
# print(a[0])
# print(a[1])
# print(a[2])
# a.reshape(-1)
# print(a)

In [0]:
# for i in a:
#     print(i)
#     print()
#     print(i.shape)
#     print()

In [0]:
# X_out = torch.tensor([])
# X_out = torch.cat((X_out, torch.tensor([1.])))
# print(X_out)

In [0]:
# import torch

# a = torch.tensor([[1, 2], [3, 4]])
# print(torch.max(a))

In [0]:
# import math
# class ts:
#     def __init__(self):
#         self.data = 1.0
#     def __log__(self):
#         print(self.x)
# t = ts()
# print(math.log(t))

In [0]:
# x = torch.tensor([1, 2, 3, -1, -2])
# print(x)

In [0]:
# from scipy import signal

# A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
# F = np.array([[1, 2], [3, 4]])
# print(signal.convolve2d(A, F))
# print(signal.correlate2d(A, F))

In [21]:
a = torch.arange(2.0 * 3.0 * 4.0).reshape(2, 3, 4)
print(a[1, ...].clone())

tensor([[12., 13., 14., 15.],
        [16., 17., 18., 19.],
        [20., 21., 22., 23.]])
